In [5]:
!wget https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/raw/main/task4/subtask_4b/subtask4b_collection_data.pkl
!wget https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/raw/main/task4/subtask_4b/subtask4b_query_tweets_dev.tsv

--2025-04-16 10:08:31--  https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/raw/main/task4/subtask_4b/subtask4b_collection_data.pkl
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16262100 (16M) [application/octet-stream]
Saving to: ‘subtask4b_collection_data.pkl’

subtask4b_collectio 100%[===================>]  15.51M  --.-KB/s    in 0.06s   

2025-04-16 10:08:31 (267 MB/s) - ‘subtask4b_collection_data.pkl’ saved [16262100/16262100]

--2025-04-16 10:08:31--  https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/raw/main/task4/subtask_4b/subtask4b_query_tweets_dev.tsv
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 298209 (291K) [text/plain]
Saving to

In [14]:
import pandas as pd
import pickle

# Load the document base
with open('subtask4b_collection_data.pkl', 'rb') as f:
    collection_data = pickle.load(f)

# Load the query set
query_data = pd.read_csv('subtask4b_query_tweets_dev.tsv', sep='\t')

<ipython-input-14-2f2bf7ae5736>:6: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  collection_data = pickle.load(f)


In [15]:
query_data.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Extract document texts from the collection data
doc_texts = [data['title'] + ' ' + data['abstract'] for _, data in collection_data.iterrows()]

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Fit the vectorizer to the document texts
doc_matrix = vectorizer.fit_transform(doc_texts)

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

results = []

for index, row in query_data.iterrows():
    tweet_id = row['post_id']
    tweet_text = row['tweet_text']

    # Transform the query tweet into a vector
    query_vec = vectorizer.transform([tweet_text])

    # Calculate cosine similarity between the query and all documents
    similarities = cosine_similarity(query_vec, doc_matrix).flatten()

    # Get the indices of the top 5 most similar documents
    top_indices = np.argsort(similarities)[::-1][:5]

    # Get the document IDs corresponding to the top indices
    # Fix: Use iloc to access elements by position
    top_doc_ids = [collection_data.iloc[i]['cord_uid'] for i in top_indices]

    results.append({
        "id": tweet_id,
        "doc_ids": top_doc_ids
    })

In [17]:
import json

with open("submission.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ TF-IDF submission saved as submission.json")

✅ TF-IDF submission saved as submission.json
